In [3]:
from transformers import AutoTokenizer
model_path = '/Users/zhangyf/llm/gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [7]:
from datasets import load_dataset
dataset_path = './sst2'
dataset = load_dataset(dataset_path)
ds_train, ds_val = dataset['train'], dataset['validation']
print(ds_train[:3])

{'idx': [0, 1, 2], 'sentence': ['hide new secretions from the parental units ', 'contains no wit , only labored gags ', 'that loves its characters and communicates something rather beautiful about human nature '], 'label': [0, 0, 1]}


In [8]:
# 定义一个新的token，奖励token eos
REWARD_TOKEN_ID = tokenizer.eos_token_id
print(REWARD_TOKEN_ID)

def tokenize(batch):
    # 提取出文本内容
    outputs = tokenizer(batch['sentence'])
    # 每条数据一个评分，初始化为 0 。
    outputs['score'] = [0] * len(outputs['input_ids'])
    # 对每条数据的最后的reward token进行评分
    outputs['score_index'] = [0] * len(outputs['input_ids'])
    for i in range(len(outputs['input_ids'])):
        # 第 i 条数据的末尾添加一个 eos token，作为reward token
        outputs['input_ids'][i].append(REWARD_TOKEN_ID)
        # reward token的掩码设置为 1 。
        outputs['attention_mask'][i].append(1)
        # 正向情感的文本评分为 1 。负向情感的评分为 0 。
        outputs['score'][i] = float(batch['label'][i])
        # 对 reward token 进行评分，也就是评分的索引为 reward token 的索引。
        outputs['score_index'][i] = len(outputs['input_ids'][i]) - 1
    return outputs

map_kwargs = {
    "batched": True,
    "batch_size": 512,
    "remove_columns": ['idx', 'sentence', 'label']
}

tokenized_dataset_train = ds_train.map(tokenize, **map_kwargs)
tokenized_dataset_val = ds_val.map(tokenize, **map_kwargs)

print(tokenized_dataset_train[4])

50256
{'input_ids': [261, 262, 5290, 15827, 12, 1659, 12, 1169, 12, 1008, 9310, 35478, 20954, 262, 28303, 714, 47478, 469, 510, 220, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'score': 0.0, 'score_index': 20}


In [9]:
print(tokenized_dataset_train[8])

{'input_ids': [64, 19095, 17280, 12, 1941, 12, 727, 705, 82, 26781, 19518, 220, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'score': 0.0, 'score_index': 12}


In [10]:
tokenized_dataset_train.set_format(type='torch')
tokenized_dataset_val.set_format(type='torch')

print(tokenized_dataset_train[8])

{'input_ids': tensor([   64, 19095, 17280,    12,  1941,    12,   727,   705,    82, 26781,
        19518,   220, 50256]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'score': tensor(0.), 'score_index': tensor(12)}


In [11]:
# 删除长度小于6的样本
tokenized_dataset_train = tokenized_dataset_train.filter(lambda x: len(x['input_ids']) > 6)
tokenized_dataset_val = tokenized_dataset_val.filter(lambda x: len(x['input_ids']) > 6)

print(len(tokenized_dataset_train))

49401


In [19]:
import torch
from torch import nn
import numpy as np
from transformers import AutoModelForCausalLM

class RewardHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        # llm最后输出的隐藏层的维度
        self.hidden_size = config.hidden_size
        # 线性层用来对llm最后输出的隐藏层给奖励
        self.reward = nn.Linear(self.hidden_size, 1)
        self._post_init()

    def _post_init(self):
        # 使用正态分布初始化权重
        nn.init.normal_(
            self.reward.weight,
            std=(1.0 / np.sqrt(self.hidden_size + 1))
        )
        # 将偏置初始化为0
        nn.init.zeros_(self.reward.bias)

    def forward(self, hidden_states):
        # 给出奖励
        return self.reward(hidden_states)

class GPT2RewardHead(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.llm = AutoModelForCausalLM.from_pretrained(model_name)
        self.reward_head = RewardHead(self.llm.config)

    def forward(self, input_ids, attention_mask):
        transformer_outputs = self.llm.forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        last_hidden_state = transformer_outputs.hidden_states[-1]
        # 给出奖励
        reward = self.reward_head(last_hidden_state).squeeze(-1)
        # sigmoid用来将奖励搞到(0,1)范围内
        return torch.sigmoid(reward)

In [20]:
model = GPT2RewardHead(model_path)

In [21]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
# 还是将 eos token 作为 pad token
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorWithPadding(tokenizer)
dataloader_params = {
    'batch_size': 32,
    'shuffle': True,
    'collate_fn': data_collator
}
train_dataloader = DataLoader(tokenized_dataset_train, **dataloader_params)
val_dataloader = DataLoader(tokenized_dataset_val, **dataloader_params)

batch = next(iter(train_dataloader))
# print(batch.keys())

print(batch['input_ids'][1])
print(batch['attention_mask'][1])
print(batch['score'][1])
print(batch['score_index'][1])
print(tokenizer.decode(batch['input_ids'][1]))
print(batch['attention_mask'][1].nonzero()[-1])

tensor([  361,  1801,   261,   290,  1527, 40667,  2230,  1194,  1628,  4077,
         2971,   837,  1306,   640,   503,   484,  1244,  1949,  5989,  1342,
         3241,   284,   262,   949,  5847,   444,   290,   517,  3241,   284,
          262,  2646,   340,   318,   546,   764,   220, 50256])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor(0.)
tensor(37)
if damon and affleck attempt another project greenlight , next time out they might try paying less attention to the miniseries and more attention to the film it is about . <|endoftext|>
tensor([37])


In [22]:
outputs = model(batch['input_ids'], batch['attention_mask'])
print(outputs.shape)

torch.Size([32, 38])


In [23]:
device = torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# 二分类交叉熵损失
criterion = nn.BCELoss()
num_epochs = 1 # N+ Implementation Detail paper

In [24]:
def validate():
    model.eval()
    total_loss = 0
    for i, batch in enumerate(val_dataloader):
        inputs = batch.to(device)
        model_inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }
        with torch.no_grad():
            # 对输出进行评分
            scores = model(**model_inputs)
            # 批次中每条数据的索引
            batch_indices = torch.arange(scores.shape[0])
            # 根据索引拿出评分，也就是reward token的评分
            score = scores[batch_indices, inputs['score_index']]
            # 目标评分，0 或者 1 。
            target = inputs['score']
            # 计算误差
            loss = criterion(score, target)
        total_loss += loss.item()
    print('validation loss:', total_loss / len(val_dataloader))

In [25]:
model.to(device)

validate()
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_dataloader):
        inputs = batch.to(device)
        model_inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }
        scores = model(**model_inputs)
        batch_indices = torch.arange(scores.shape[0])
        score = scores[batch_indices, inputs['score_index']]
        target = inputs['score']
        loss = criterion(score, target)
        # 三部曲：清空梯度 ⟶ 反向传播计算梯度 ⟶ 更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("损失:",loss.item())
    validate()

validation loss: 2.7581602888447896
3.108236789703369
0.8091529607772827
1.4204561710357666
1.0424749851226807
0.8312817811965942
0.7022022604942322
0.8008676767349243
0.9361941814422607
1.125940203666687
0.9442117214202881
0.7852380275726318
0.6829521059989929
0.598456621170044
0.7150830030441284
0.6870534420013428
0.7926227450370789
0.9951375126838684
0.8767225742340088
0.9890028834342957
0.7926579713821411
0.7238250970840454
0.7270512580871582
0.6529476642608643
0.7587059736251831
0.759275496006012
0.7556922435760498
0.7033265829086304
0.6928670406341553
0.7024922370910645
0.8076363205909729
0.7934281826019287
0.766585111618042
0.7283234596252441
0.7111377716064453
0.6854632496833801
0.701343297958374
0.6488975286483765
0.6917641162872314
0.7289260625839233
0.7580981254577637
0.7184112071990967
0.6983562707901001
0.7334611415863037
0.6928342580795288
0.7089872360229492
0.6673755049705505
0.6703251600265503
0.6563602685928345
0.6812747716903687
0.6613168716430664
0.6867802143096924
0

KeyboardInterrupt: 

In [20]:
torch.save(model.state_dict(), 'reward_model.pt')

In [21]:
validate()

validation loss: 0.3069670636136185


In [26]:
from sklearn.metrics import confusion_matrix
model.eval()

all_predictions = []
all_labels = []

for i, batch in enumerate(val_dataloader):
    inputs = batch.to(device)
    model_inputs = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }
    with torch.no_grad():
        scores = model(**model_inputs)
        batch_indices = torch.arange(scores.shape[0])
        score = scores[batch_indices, inputs['score_index']]
        target = inputs['score']
    predictions = (score > 0.5).int()

    all_predictions.extend(predictions.cpu().numpy())
    all_labels.extend(target.cpu().numpy())

confusion_matrix(all_labels, all_predictions)

array([[364,  60],
       [ 43, 400]])